In [139]:
import geojson
from geojson import dump
import json
import shapely
from shapely import Point
from pyproj import Transformer

# load lighthouse features,
# make GeoJSON template,
# and define transformer

with open("histLightReproj.geojson") as f:
    lh=geojson.load(f)
features=lh['features']
GeoJson= {"type": "FeatureCollection",
          "name": "lighthouseBuffers",
          "features": []
          }
transformer = Transformer.from_crs(4326, 27700, always_xy=True)

# loop through each lighthouse record,
# buffer those records with a `REACH_1851` value,
# and append them to the template

for f in features:
    coords=(f['geometry']['coordinates'])
    dist=(f['properties']['REACH_1851']*1609.34)
    name=(f['properties']['Name'])
    if len(json.loads(shapely.to_geojson(shapely.buffer(Point(coords), dist)))['coordinates'][0]) > 0:
        feat={
        "type": "Feature",
        "properties": {
            "Name": name,
        },
        "geometry": json.loads(shapely.to_geojson(shapely.buffer(Point(coords), dist)))
        },
        GeoJson['features'].append(feat[0])

# export new data as `buffers`

with open('buffers.geojson', 'w') as f:
   dump(GeoJson, f)


